In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir('/content/drive/MyDrive/DataCollection/tensorflow-ml-nlp-tf2/5.TEXT_SIM')

In [3]:
os.getcwd()

'/content/drive/MyDrive/DataCollection/tensorflow-ml-nlp-tf2/5.TEXT_SIM'

# XGboost

In [4]:
import pandas as pd
import numpy as np
import os

import json

from sklearn.model_selection  import train_test_split

### 설정된 값들

In [5]:
DATA_IN_PATH = './data_in/'
DATA_OUT_PATH = './data_out/'

TRAIN_Q1_DATA_FILE = 'train_q1.npy'
TRAIN_Q2_DATA_FILE = 'train_q2.npy'
TRAIN_LABEL_DATA_FILE = 'train_label.npy'


# 훈련 데이터 가져오는 부분이다.
train_q1_data = np.load(open(DATA_IN_PATH + TRAIN_Q1_DATA_FILE, 'rb'))
train_q2_data = np.load(open(DATA_IN_PATH + TRAIN_Q2_DATA_FILE, 'rb'))
train_labels = np.load(open(DATA_IN_PATH + TRAIN_LABEL_DATA_FILE, 'rb'))

In [6]:
train_input = np.stack((train_q1_data, train_q2_data), axis=1) 
# np.stack은 두 질문을 하나의 쌍으로 만든다.
# 예를 들어 질문 [A]와 질문 [B]가 있을 때, 이 질문을 하나로 묶어 [[A],[B]] 형태로 만든다.

In [7]:
print(train_input.shape)  # (전체 데이터 갯수, 1쌍, 질문 길이)

(298526, 2, 31)


### 훈련 셋과 평가 셋 나누기¶

In [8]:
train_input, eval_input, train_label, eval_label = train_test_split(train_input, train_labels, test_size=0.2, random_state=4242)

In [9]:
import xgboost as xgb

### 모델 구성

In [12]:
train_data = xgb.DMatrix(train_input.sum(axis=1), label=train_label) # 학습 데이터 읽어 오기
eval_data = xgb.DMatrix(eval_input.sum(axis=1), label=eval_label) # 평가 데이터 읽어 오기

data_list = [(train_data, 'train'), (eval_data, 'valid')]

In [10]:
train_input[0]

array([[    9,   348,    37, 23601,   316,   348, 15392,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0],
       [    9,    37,   293,   348,  6482,     8,  3312,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0]], dtype=int32)

In [11]:
train_input[0].sum(axis=1)  ##어떻게 sum이되는지 print 해보았다.

array([40051, 10489])

In [ ]:
params = {} # 인자를 통해 XGB모델에 넣어 주자 
params['objective'] = 'binary:logistic' # 목적함수인 경우 이진 로지스틱을 사용.
params['eval_metric'] = 'rmse' # root mean square error를 사용  

bst = xgb.train(params, train_data, num_boost_round = 1000, evals = data_list, early_stopping_rounds=10)
#evals : 모델 검증 시 사용할 전체 데이터 쌍

### 테스트 데이터 가져오기

In [16]:
TEST_Q1_DATA_FILE = 'test_q1.npy'
TEST_Q2_DATA_FILE = 'test_q2.npy'
TEST_ID_DATA_FILE = 'test_id.npy'

test_q1_data = np.load(open(DATA_IN_PATH + TEST_Q1_DATA_FILE, 'rb'))
test_q2_data = np.load(open(DATA_IN_PATH + TEST_Q2_DATA_FILE, 'rb'))
# test_id_data = np.load(open(DATA_IN_PATH + TEST_ID_DATA_FILE, 'rb'))

### 예측하기

In [15]:
test_input = np.stack((test_q1_data, test_q2_data), axis=1) 
test_data = xgb.DMatrix(test_input.sum(axis=1))
test_predict = bst.predict(test_data)

In [17]:
## test_id_data가 안 불러져서, 그냥 test_predict로만 dataframe을 만들어본다.
output_sample = pd.DataFrame({'is_duplicate': test_predict})

In [18]:
output_sample.head(10)  ## 이 값이 곧 유사도(확률)인듯.(1이 duplicate로 supervised learning 했으니까)

,is_duplicate
0,0.384445
1,0.572531
2,0.797910
3,0.039513
4,0.638644
5,0.069173
6,0.874005
7,0.346784
8,0.804683
9,0.453130


In [ ]:
if not os.path.exists(DATA_OUT_PATH):
    os.makedirs(DATA_OUT_PATH)
    
output = pd.DataFrame({'test_id': test_id_data, 'is_duplicate': test_predict})
output.to_csv(DATA_OUT_PATH + 'simple_xgb.csv', index=False)

In [ ]:
 # TF-IDF나 word2vec으로 데이터의 입력값의 형태를 바꾼 후 모델을 적용하면 성능이 더 좋아질듯.
 ## 현재는 그냥 vocab index에 sum 한 값을 넣어줘서 성능이 안 좋은듯.